In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_us.model_api import *
import pandas as pd
import numpy as np

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sim = Microsimulation(dataset="hf://policyengine/policyengine-us-data/pooled_3_year_cps_2023.h5")

In [3]:
# Get CA household data
ca_mask = sim.calculate("state_code_str", map_to="household") == "CA"
income = sim.calculate("adjusted_gross_income", map_to="household")[ca_mask]
poverty_threshold = sim.calculate("tax_unit_fpg", map_to="household")[ca_mask]

print(f"CA households: {ca_mask.sum()/1e6:.1f}M")

CA households: 14.2M


In [ ]:
def create_aei_reform():
    class ca_aei_rebate(Variable):
        value_type = float
        entity = Household
        label = "California AEI rebate"
        unit = USD
        definition_period = YEAR

        def formula(household, period, parameters):
            income = household("adjusted_gross_income", period)
            fpg = household("tax_unit_fpg", period)
            income_to_fpg_ratio = where(fpg > 0, income / fpg, np.inf)
            
            # Phase-out parameters
            phaseout_start = 1.5   # 150% FPG
            phaseout_end = 1.75    # 175% FPG
            phaseout_width = phaseout_end - phaseout_start
            
            # Phase-out calculation (similar to ME structure)
            excess = max_(income_to_fpg_ratio - phaseout_start, 0)
            phaseout_percentage = min_(1, excess / phaseout_width)
            
            return where(
                income_to_fpg_ratio <= phaseout_end,
                fpg * (1 - phaseout_percentage),
                0
            )

    class reform(Reform):
        def apply(self):
            self.add_variable(ca_aei_rebate)
    
    return reform

# Apply reform and calculate
reform = create_aei_reform()
sim_reform = Microsimulation(dataset="hf://policyengine/policyengine-us-data/pooled_3_year_cps_2023.h5", reform=reform)
ca_mask_reform = sim_reform.calculate("state_code_str", map_to="household") == "CA"
X = sim_reform.calculate("ca_aei_rebate")[ca_mask_reform].sum()

print(f"X (tax-free consumption): ${X/1e9:.1f}B")

In [ ]:
# Summary results
rebates = sim_reform.calculate("ca_aei_rebate")[ca_mask_reform]
households_with_rebates = (rebates > 0).sum()
total_households = ca_mask_reform.sum()

print(f"Total CA households: {total_households/1e6:.1f}M")
print(f"Households with rebates: {households_with_rebates/1e6:.1f}M ({100*households_with_rebates/total_households:.0f}%)")
print(f"Average rebate: ${rebates[rebates > 0].mean():,.0f}")
print(f"X = ${X:,.0f}")